### 1.	Deep Learning.
a.	Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.


In [ ]:
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

he_init = tf.contrib.layers.variance_scaling_initializer()

In [ ]:
from functools import partial

dense_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=he_init)

In [ ]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

with tf.name_scope('dnn'):
    hidden1 = dense_layer(X, n_hidden1, name='hidden1')
    hidden2 = dense_layer(hidden1, n_hidden2, name='hidden2')
    hidden3 = dense_layer(hidden2, n_hidden3, name='hidden3')
    hidden4 = dense_layer(hidden3, n_hidden4, name='hidden4')
    hidden5 = dense_layer(hidden4, n_hidden5, name='hidden5')
    logits = dense_layer(hidden5, n_outputs, activation=None, name='outputs')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

b.	Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.

In [ ]:
learning_rate = 0.001

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

In [ ]:
X_train = mnist.train.images[mnist.train.labels < 5]
y_train = mnist.train.labels[mnist.train.labels < 5]
X_test = mnist.test.images[mnist.test.labels < 5]
y_test = mnist.test.labels[mnist.test.labels < 5]

In [ ]:
def shuffle_split(X, y, n_batches):
    np.random.seed(seed=42)
    rnd_idx = np.random.permutation(len(X))
    for i_idx in np.array_split(rnd_idx, n_batches):
        X_batch = X[i_idx]
        y_batch = y[i_idx]
        yield X_batch, y_batch


In [ ]:
n_epochs = 50
batch_size = 50
n_batches = len(X_train) // batch_size
best_loss = float('inf')
patience = 2
cnt_patience = 0

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_split(X_train, y_train, n_batches):
            sess.run([training_op, loss], feed_dict={X: X_batch, y: y_batch})
        accuracy_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        accuracy_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        loss_test = loss.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, 'loss', loss_test, 'accuracy_train:', accuracy_train, 'accuracy_test:', accuracy_test)
        if loss_test < best_loss:
            best_loss = loss_test
        else:
            cnt_patience += 1
            if cnt_patience > patience:
                'Early stopping!'
                break

c.	Tune the hyperparameters using cross-validation and see what precision you can achieve.
d.	Now try adding Batch Normalization and compare the learning curves: is it converging faster than before? Does it produce a better model?
e.	Is the model overfitting the training set? Try adding dropout to every layer and try again. Does it help?


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
import tensorflow as tf
from functools import partial
import numpy as np


class DnnClassifier(BaseEstimator, ClassifierMixin):
    
    
    def __init__(self, 
                 batch_size=50, 
                 n_neuron=100):
        
        self.batch_size = batch_size
        self.n_neuron = n_neuron
        
    def reset_graph(self, seed=42):
        tf.reset_default_graph()
        tf.set_random_seed(seed)
        np.random.seed(seed)
        
    def _build_graph(self, n_inputs, n_outputs):
        
        self.reset_graph()
        
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
        y = tf.placeholder(tf.int64, shape=(None), name='y')
        
        he_init = tf.contrib.layers.variance_scaling_initializer()
        
        dense_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=he_init)
with tf.name_scope('dnn'):
            hidden1 = dense_layer(X, self.n_neuron, name='hidden1')
            hidden2 = dense_layer(hidden1, self.n_neuron, name='hidden2')
            hidden3 = dense_layer(hidden2, self.n_neuron, name='hidden3')
            hidden4 = dense_layer(hidden3, self.n_neuron, name='hidden4')
            hidden5 = dense_layer(hidden4, self.n_neuron, name='hidden5')
            logits = dense_layer(hidden5, n_outputs, activation=None, name='outputs')
            
        with tf.name_scope('softmax'):
            y_proba = tf.nn.softmax(logits, axis=1, name='y_proba')
    
        with tf.name_scope('loss'):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
            loss = tf.reduce_mean(xentropy, name='loss')
        
        learning_rate = 0.001

        with tf.name_scope('train'):
            optimizer = tf.train.AdamOptimizer(learning_rate)
            training_op = optimizer.minimize(loss)
            
        with tf.name_scope('eval'):
            correct = tf.nn.in_top_k(logits, y, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
            
        self._training_op = training_op
        self._accuracy = accuracy
        self._X = X
        self._y = y
        self._logits = logits
        self._y_proba = y_proba
         def shuffle_split(self, X, y, n_batches):
        np.random.seed(seed=42)
        rnd_idx = np.random.permutation(len(X))
        for i_idx in np.array_split(rnd_idx, n_batches):
            X_batch = X[i_idx]
            y_batch = y[i_idx]
            yield X_batch, y_batch
        
    def fit(self, X, y, n_epochs=5, X_valid=None, y_valid=None):
        self.n_batches = len(X) // self.batch_size
        
        n_inputs = X.shape[1]
        self.n_outputs = len(np.unique(y))
        self.classes_ = np.unique(y)
        
        self._build_graph(n_inputs, self.n_outputs)
        
        init = tf.global_variables_initializer()

        self._session = tf.Session()
        with self._session.as_default() as sess:
            init.run()
            for epoch in range(n_epochs):
                for X_batch, y_batch in self.shuffle_split(X, y, self.n_batches):
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    sess.run([self._training_op], feed_dict=feed_dict)
                accuracy_train = self._accuracy.eval(feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    accuracy_valid = self._accuracy.eval(feed_dict={self._X: X_valid, self._y: y_valid})
                    print('epoch:', epoch, 'accuracy_valid:', accuracy_valid*100)
                else:
                    print('epoch:', epoch, 'accuracy_train:', accuracy_train*100)
                
                
        return self
    
     def predict_proba(self, X):
        with self._session.as_default() as sess:
            y_proba = sess.run([self._y_proba], feed_dict={self._X: X})
            return np.array(y_proba).reshape((len(X), self.n_outputs))
        
    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

X_train = mnist.train.images
y_train = mnist.train.labels
X_valid = mnist.test.images
y_valid = mnist.test.labels

In [ ]:
dnn_clf = DnnClassifier()
dnn_clf.fit(X_train, y_train, 20, X_valid, y_valid)

In [ ]:
from sklearn.grid_search import GridSearchCV

params_grid = [{'batch_size': [20, 50]}, {'n_neuron': [50, 100]}]
gs = GridSearchCV(DnnClassifier(), params_grid, 
                 fit_params={'n_epochs': 20, 'X_valid': X_valid, 'y_valid': y_valid})

gs.fit(X_train, y_train)

gs.best_params_

### 2.	Transfer learning.
a.	Create a new DNN that reuses all the pretrained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a new one.
b.	Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision?
c.	Try caching the frozen layers, and train the model again: how much faster is it now?
d.	Try again reusing just four hidden layers instead of five. Can you achieve a higher precision?
e.	Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?


### 3.	Pretraining on an auxiliary task.
a.	In this exercise you will build a DNN that compares two MNIST digit images and predicts whether they represent the same digit or not. Then you will reuse the lower layers of this network to train an MNIST classifier using very little training data. Start by building two DNNs (let’s call them DNN A and B), both similar to the one you built earlier but without the output layer: each DNN should have five hidden layers of 100 neurons each, He initialization, and ELU activation. Next, add one more hidden layer with 10 units on top of both DNNs. To do this, you should use TensorFlow’s concat() function with axis=1 to concatenate the outputs of both DNNs for each instance, then feed the result to the hidden layer. Finally, add an output layer with a single neuron using the logistic activation function.
b.	Split the MNIST training set in two sets: split #1 should containing 55,000 images, and split #2 should contain contain 5,000 images. Create a function that generates a training batch where each instance is a pair of MNIST images picked from split #1. Half of the training instances should be pairs of images that belong to the same class, while the other half should be images from different classes. For each pair, the training label should be 0 if the images are from the same class, or 1 if they are from different classes.
c.	Train the DNN on this training set. For each image pair, you can simultaneously feed the first image to DNN A and the second image to DNN B. The whole network will gradually learn to tell whether two images belong to the same class or not.
d.	Now create a new DNN by reusing and freezing the hidden layers of DNN A and adding a softmax output layer on top with 10 neurons. Train this network on split #2 and see if you can achieve high performance despite having only 500 images per class.

